In [ ]:
# --- ランダムフォレストモデルの再作成・保存に特化したコード ---
import pandas as pd
import glob
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive

# Driveマウントと準備
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/master_research'
os.chdir(project_path)

# データの読み込み
print("--- データの準備中... ---")
features_directory = 'features_landmarks'
train_files = sorted(glob.glob(os.path.join(features_directory, '*.csv')))[:12]
train_df = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)

# データの前処理
feature_columns = ['torso_angle_deg', 'travel_angle_smooth', 'speed_smooth', 'torso_angular_velocity_smooth', 'travel_angular_velocity_smooth', 'speed_acceleration_smooth']
label_column = 'phase_label'
columns_to_check_for_nan = feature_columns + [label_column]
train_df.dropna(subset=columns_to_check_for_nan, inplace=True)
X_train = train_df[feature_columns]
y_train = train_df[label_column]
print("--- データ準備完了 ---")

# モデルの訓練
print("\n--- ランダムフォレストモデルの訓練を開始 ---")
model_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_rf.fit(X_train, y_train)
print("--- 訓練完了 ---")

# モデルの保存
joblib.dump(model_rf, 'random_forest_model.joblib')
print(f"\n>>> モデルタイプ '{type(model_rf)}' を 'random_forest_model.joblib' として正しく保存しました。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- データの準備中... ---
--- データ準備完了 ---

--- ランダムフォレストモデルの訓練を開始 ---
--- 訓練完了 ---

>>> モデルタイプ '<class 'sklearn.ensemble._forest.RandomForestClassifier'>' を 'random_forest_model.joblib' として正しく保存しました。


In [ ]:
# --- ランダムフォレスト単体の動作確認 ---
import pandas as pd
import glob
import os
import joblib
from sklearn.metrics import classification_report
from google.colab import drive

# Driveマウントと準備
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/master_research'
os.chdir(project_path)

print("--- RFモデルとデータの読み込み ---")
try:
    model_rf = joblib.load('random_forest_model.joblib')
    print(f"ロードしたモデルのタイプ: {type(model_rf)}")

    test_files = sorted(glob.glob('features_landmarks/landmarks_0001[3-5]_features.csv'))
    test_df = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True)

    feature_columns = ['torso_angle_deg', 'travel_angle_smooth', 'speed_smooth', 'torso_angular_velocity_smooth', 'travel_angular_velocity_smooth', 'speed_acceleration_smooth']
    label_column = 'phase_label'

    test_df.dropna(subset=feature_columns + [label_column], inplace=True)

    X_test_flat = test_df[feature_columns]
    y_test = test_df[label_column]

    print("\n--- RFモデルの予測を実行 ---")
    y_pred_rf = model_rf.predict(X_test_flat)

    print("\n--- RFモデルの評価レポート ---")
    print(classification_report(y_test, y_pred_rf, target_names=['1: 直線', '2: ターン']))
    print("\n>>> RFモデル単体のテストは正常に完了しました。")

except Exception as e:
    print(f"\nエラーが発生しました: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- RFモデルとデータの読み込み ---
ロードしたモデルのタイプ: <class 'keras.src.models.sequential.Sequential'>

--- RFモデルの予測を実行 ---

エラーが発生しました: Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", shape=(32, 6), dtype=float32). Expected shape (None, 100, 6), but input has incompatible shape (32, 6)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 6), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# --- LSTM単体の動作確認 ---
import pandas as pd
import numpy as np
import glob
import os
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Driveマウントと準備（再度実行しても問題ありません）
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/master_research'
os.chdir(project_path)


print("--- LSTMモデルとデータの読み込み ---")
try:
    # Kerasの警告を非表示にする
    tf.get_logger().setLevel('ERROR')

    model_lstm = load_model('lstm_model.h5')
    scaler = joblib.load('scaler.joblib')
    print(f"ロードしたモデルのタイプ: {type(model_lstm)}")

    test_files = sorted(glob.glob('features_landmarks/landmarks_0001[3-5]_features.csv'))
    test_df = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True)

    feature_columns = ['torso_angle_deg', 'travel_angle_smooth', 'speed_smooth', 'torso_angular_velocity_smooth', 'travel_angular_velocity_smooth', 'speed_acceleration_smooth']
    label_column = 'phase_label'
    test_df.dropna(subset=feature_columns + [label_column], inplace=True)

    X_test_flat = test_df[feature_columns]
    y_test = test_df[label_column]
    X_test_scaled = scaler.transform(X_test_flat)

    TIME_STEPS = 100
    def create_sequences(X, y, time_steps=TIME_STEPS):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:(i + time_steps)])
            ys.append(y.iloc[i + time_steps])
        return np.array(Xs), np.array(ys)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test - 1)

    print("\n--- LSTMモデルの予測を実行 ---")
    y_pred_proba_lstm = model_lstm.predict(X_test_seq)
    y_pred_lstm = (y_pred_proba_lstm > 0.5).astype(int) + 1

    print("\n--- LSTMモデルの評価レポート ---")
    print(classification_report(y_test_seq + 1, y_pred_lstm, target_names=['1: 直線', '2: ターン']))
    print("\n>>> LSTMモデル単体のテストは正常に完了しました。")

except Exception as e:
    print(f"\nエラーが発生しました: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- LSTMモデルとデータの読み込み ---
ロードしたモデルのタイプ: <class 'keras.src.models.sequential.Sequential'>

--- LSTMモデルの予測を実行 ---
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

--- LSTMモデルの評価レポート ---
              precision    recall  f1-score   support

       1: 直線       0.67      0.83      0.74       697
      2: ターン       0.78      0.60      0.68       704

    accuracy                           0.71      1401
   macro avg       0.73      0.71      0.71      1401
weighted avg       0.73      0.71      0.71      1401


>>> LSTMモデル単体のテストは正常に完了しました。


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import time
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score, accuracy_score
import warnings

# Kerasの警告を非表示にする
warnings.filterwarnings("ignore", category=UserWarning, module='keras')
tf.get_logger().setLevel('ERROR')

# --- 1. 準備：Driveマウントとモデル・データの読み込み ---
print("--- ステップ1: 準備 ---")
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/master_research'
os.chdir(project_path)

try:
    model_rf = joblib.load('random_forest_model.joblib')
    model_lstm = load_model('lstm_model.h5')
    scaler = joblib.load('scaler.joblib')
    print("モデルとスケーラーの読み込みに成功しました。")
    print(f"  - RFモデルのタイプ: {type(model_rf)}")
    print(f"  - LSTMモデルのタイプ: {type(model_lstm)}")
except FileNotFoundError as e:
    print(f"エラー: モデルファイルが見つかりません。 {e}")
    print("事前準備として、モデルの保存が正しく完了しているか確認してください。")

# テストデータの準備
test_files = sorted(glob.glob('features_landmarks/landmarks_0001[3-5]_features.csv'))
test_df = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True)

feature_columns = ['torso_angle_deg', 'travel_angle_smooth', 'speed_smooth', 'torso_angular_velocity_smooth', 'travel_angular_velocity_smooth', 'speed_acceleration_smooth']
label_column = 'phase_label'
test_df.dropna(subset=feature_columns + [label_column], inplace=True)

X_test_flat = test_df[feature_columns]
y_test = test_df[label_column]
X_test_scaled = scaler.transform(X_test_flat)

TIME_STEPS = 100
def create_sequences(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test - 1)

print("--- 準備完了 ---\n")

# --- 2. 各モデルの性能評価 ---
results = {}

# --- A) 高速モデル（RF）のみの評価 ---
print("--- A) 高速モデル（ランダムフォレスト）の評価中... ---")
start_time = time.time()
y_pred_rf = model_rf.predict(X_test_flat)
end_time = time.time()
avg_time_rf = ((end_time - start_time) / len(X_test_flat)) * 1000
results['高速モデル (RF Only)'] = {
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'f1_turn': f1_score(y_test, y_pred_rf, pos_label=2),
    'time_ms': avg_time_rf
}

# --- B) 高精度モデル（LSTM）のみの評価 ---
print("--- B) 高精度モデル（LSTM）の評価中... ---")
start_time = time.time()
y_pred_proba_lstm = model_lstm.predict(X_test_seq)
y_pred_lstm = (y_pred_proba_lstm > 0.5).astype(int) + 1
end_time = time.time()
avg_time_lstm = ((end_time - start_time) / len(X_test_seq)) * 1000
results['高精度モデル (LSTM Only)'] = {
    'accuracy': accuracy_score(y_test_seq + 1, y_pred_lstm),
    'f1_turn': f1_score(y_test_seq + 1, y_pred_lstm, pos_label=2),
    'time_ms': avg_time_lstm
}

# --- C) ハイブリッドモデルの評価 ---
print("--- C) ハイブリッドモデルの評価中... ---")
threshold = 0.85
y_pred_hybrid = []
lstm_invocations = 0

start_time = time.time()
for i in range(len(test_df)):
    rf_input = X_test_flat.iloc[i].values.reshape(1, -1)
    rf_proba = model_rf.predict_proba(rf_input)
    confidence = np.max(rf_proba)

    if confidence >= threshold or i < TIME_STEPS:
        prediction = np.argmax(rf_proba, axis=1)[0] + 1
    else:
        lstm_invocations += 1
        lstm_input = X_test_scaled[i-TIME_STEPS:i].reshape(1, TIME_STEPS, len(feature_columns))
        lstm_proba = model_lstm.predict(lstm_input, verbose=0)
        prediction = (lstm_proba > 0.5).astype(int)[0][0] + 1

    y_pred_hybrid.append(prediction)
end_time = time.time()

avg_time_hybrid = ((end_time - start_time) / len(test_df)) * 1000
results['ハイブリッドモデル'] = {
    'accuracy': accuracy_score(y_test, y_pred_hybrid),
    'f1_turn': f1_score(y_test, y_pred_hybrid, pos_label=2),
    'time_ms': avg_time_hybrid
}
print(f"高精度モデル（LSTM）の起動回数: {lstm_invocations} / {len(test_df)} フレーム ({lstm_invocations/len(test_df)*100:.1f}%)")

# --- 3. 最終結果のサマリー ---
print("\n\n---【最終性能比較サマリー】---")
print(f"{'モデル名':<20} | {'全体の正解率':<12} | {'方向転換のf1-score':<18} | {'平均処理時間(ms/フレーム)':<22}")
print("-" * 80)
for model_name, result in results.items():
    print(f"{model_name:<20} | {result['accuracy'] * 100:^12.2f}% | {result['f1_turn']:^18.4f} | {result['time_ms']:.4f}")

--- ステップ1: 準備 ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


モデルとスケーラーの読み込みに成功しました。
  - RFモデルのタイプ: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
  - LSTMモデルのタイプ: <class 'keras.src.models.sequential.Sequential'>
--- 準備完了 ---

--- A) 高速モデル（ランダムフォレスト）の評価中... ---
--- B) 高精度モデル（LSTM）の評価中... ---
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
--- C) ハイブリッドモデルの評価中... ---


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local

高精度モデル（LSTM）の起動回数: 632 / 1501 フレーム (42.1%)


---【最終性能比較サマリー】---
モデル名                 | 全体の正解率       | 方向転換のf1-score      | 平均処理時間(ms/フレーム)       
--------------------------------------------------------------------------------
高速モデル (RF Only)      |    68.62    % |       0.5901       | 0.0315
高精度モデル (LSTM Only)   |    71.31    % |       0.6774       | 0.8549
ハイブリッドモデル            |    70.69    % |       0.6469       | 83.4288


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
